Import all

In [ ]:
pip install nibabel numpy scipy matplotlib scikit-image torch torchvision pandas

GPU Available?

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

GoogLeNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 1
EPOCHS = 30
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0) 
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

# Arqquitectura InceptionV3
class InceptionV3Block3D(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1 = nn.Conv3d(in_channels, 64, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv3d(in_channels, 48, kernel_size=1),
            nn.ReLU(),
            nn.Conv3d(48, 64, kernel_size=5, padding=2)
        )

        self.branch3 = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=1),
            nn.ReLU(),
            nn.Conv3d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(96, 96, kernel_size=3, padding=1)
        )

        self.branch4 = nn.Sequential(
            nn.AvgPool3d(kernel_size=3, stride=1, padding=1),
            nn.Conv3d(in_channels, 32, kernel_size=1)
        )

    def forward(self, x):
        return torch.cat([
            self.branch1(x),
            self.branch2(x),
            self.branch3(x),
            self.branch4(x)
        ], 1)

#Arquitectura GoogLeNetV3
class GoogLeNetV3_3D(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv3d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )

        self.inception_a1 = InceptionV3Block3D(64)
        self.inception_a2 = InceptionV3Block3D(224)
        self.inception_a3 = InceptionV3Block3D(224)

        self.reduction_a = nn.Sequential(
            nn.Conv3d(224, 384, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )

        self.inception_b1 = InceptionV3Block3D(384)
        self.inception_b2 = InceptionV3Block3D(224)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(224, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_a1(x)
        x = self.inception_a2(x)
        x = self.inception_a3(x)
        x = self.reduction_a(x)
        x = self.inception_b1(x)
        x = self.inception_b2(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)

#Cargar datos
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Enrenamiento del modelo
model = GoogLeNetV3_3D(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)
    print(f"Test Acc: {acc_val:.2f}%")

# ========= GUARDAR =========
torch.save(model.state_dict(), "googlenetv3_3d_alzheimer.pth")
print("Modelo guardado como googlenetv3_3d_alzheimer.pth")


Entrenar mas epocas

In [ ]:

import os
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import scipy.ndimage

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv" 
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 2
EPOCHS = 5  
START_EPOCH = 75 
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)
        volume = torch.tensor(volume).unsqueeze(0)
        label = torch.tensor(self.label_encoder.transform([label])[0])

        return volume, label

class GoogLeNet3D(nn.Module):
    def __init__(self, num_classes):
        super(GoogLeNet3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=7, stride=2, padding=3),  
            nn.ReLU(inplace=True),
            nn.MaxPool3d(3, stride=2, padding=1),                 
            nn.Conv3d(64, 128, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(3, stride=2, padding=1),                  
            nn.Conv3d(192, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool3d((1, 1, 1))                        
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Cargar datos
df = pd.read_csv(CSV_PATH) 
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

model = GoogLeNet3D(num_classes=len(le.classes_)).to(DEVICE)
model.load_state_dict(torch.load(r"D:\ADNI Data/googlenet3d_alzheimer_extended3.pth"))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(START_EPOCH, START_EPOCH + EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{START_EPOCH + EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    # Validación
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)
    print(f"Test Acc: {acc_val:.2f}%")

# ========= GUARDAR MODELO Y CHECKPOINT
torch.save(model.state_dict(), "googlenet3d_alzheimer_extended4.pth")
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'googlenet3d_checkpoint_epoch80.pth')
print("Modelo actualizado y checkpoint guardado.")
